# Aux

In [22]:
import torch
import torch.optim as optim
#%pip install torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, TensorDataset
import numpy as np

#
import torch
import torch.nn as nn
import torch.optim as optim

#%pip install pennylane --user
import pennylane as qml
from pennylane import numpy as np

import sys
import os

src_path = os.path.abspath(os.path.join('..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

from circuits.convolution.default import default_circuit, custom_circuit
from circuits.convolution.NQ_circuit import NQ_circuit
from circuits.convolution.no_entanglement_circuit import no_entanglement_random_circuit
from circuits.embedding.default import default_embedding, custom_embedding
from circuits.embedding.IQP_embedding import custom_iqp_embedding
from circuits.embedding.NQE_embedding import NQE_embedding
from circuits.embedding.qml_embeddings import angle_embedding, amplitude_embedding, QAOA_embedding, displacement_embedding, squeezing_embedding
from circuits.measurement.default import default_measurement, custom_measurement
from layers.quanvolution import QuanvLayer
from models.quanvolutional_net import QuanvolutionalNet
from utils.data_loader import load_mnist_data

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Train

In [27]:
#%pip install qcml
#%pip install scikit-learn
#%pip install dask
#%pip install pynvml
#%pip install pandas
#%pip install matplotlib --user
#%pip install seaborn
#%pip install jax
from qcml.bench.grid_search import GridSearch
from qcml.utils.log import log_setup
import os

log_settings = {
    "output": "both",
    "terminal_level": "INFO",
    "file_level": "DEBUG",
    "hide_logs": ["jax", "pennylane", "bokeh", "distributed"],
    "slack_notify": False
}

log_setup(**log_settings)

X_train, y_train, X_val, y_val = load_mnist_data(batch_size=4, output='np', limit=250)

os.environ["JAX_PLATFORM_NAME"] = "cpu"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
torch.backends.cudnn.benchmark = True

2024-10-03 10:50:14,677 - root - INFO - Logging is set up. (log.py:391)
2024-10-03 10:50:14,678 - root - INFO - jax logs are set to ERROR level. (log.py:421)
2024-10-03 10:50:14,679 - root - INFO - pennylane logs are set to ERROR level. (log.py:421)
2024-10-03 10:50:14,681 - root - INFO - bokeh logs are set to ERROR level. (log.py:421)
2024-10-03 10:50:14,683 - root - INFO - distributed logs are set to ERROR level. (log.py:421)


Using device: cpu


In [29]:
import itertools

embedding_type = "IQP"

qkernel_shape = [2, 3]
classical_kernel_shape = [2, 3, 5]

if embedding_type == "angle":

    # Angle embedding

    embedding = angle_embedding
    circuit = no_entanglement_random_circuit
    params = {}

    lrs = torch.linspace(0.009, 0.08, 3, requires_grad=False)
    learning_rates = [lr.item() for lr in lrs] 
    

elif embedding_type == "amplitude":

    # Amplitude embedding

    embedding = amplitude_embedding
    circuit = no_entanglement_random_circuit
    params = {}

    lrs = torch.linspace(0.008, 0.03, 3, requires_grad=False)
    learning_rates = [lr.item() for lr in lrs]
    

elif embedding_type == "IQP":

    # IQP embedding

    embedding = custom_iqp_embedding
    circuit = NQ_circuit
    params = {"embedding": {"n_repeats": 2}}

    lrs = torch.linspace(0.008, 0.05, 3, requires_grad=False)
    learning_rates = [lr.item() for lr in lrs]

elif embedding_type == "NQE":

    # NQE embedding

    embedding = NQE_embedding
    circuit = no_entanglement_random_circuit
    params = {"embedding": {"n_repeats": 2}}

    lrs = torch.linspace(0.008, 0.05, 3, requires_grad=False)
    learning_rates = [lr.item() for lr in lrs]

elif embedding_type == "displacement":

    # Displacement embedding

    embedding = angle_embedding
    circuit = no_entanglement_random_circuit
    params = {}

    lrs = torch.linspace(0.009, 0.08, 3, requires_grad=False)
    learning_rates = [lr.item() for lr in lrs]

elif embedding_type == "squeezing":

    # Squeezing embedding

    embedding = angle_embedding
    circuit = no_entanglement_random_circuit
    params = {}

    lrs = torch.linspace(0.009, 0.08, 3, requires_grad=False)
    learning_rates = [lr.item() for lr in lrs]

elif embedding_type == "QAOA":

    # Squeezing embedding

    embedding = QAOA_embedding
    circuit = NQ_circuit
    params = {}

    lrs = torch.linspace(0.009, 0.03, 3, requires_grad=False)
    learning_rates = [lr.item() for lr in lrs]

model_grid = {
        "qkernel_shape": qkernel_shape,
        "classical_kernel_shape": classical_kernel_shape,
        "embedding":[embedding],
        "circuit":[circuit],
        "measurement":[custom_measurement],
        "params":[params],
        "n_classes":[10],
        "learning_rate": learning_rates
    }

experiment_name = "quantum_mnist_" + embedding_type + ","

gs = GridSearch(
    classifiers=[QuanvolutionalNet],
    param_grid=model_grid,
    #combinations=combinations,
    batch_size=50,
    experiment_name=experiment_name
).run(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val)

2024-10-03 11:03:15,575 - qcml.bench.grid_search - INFO - Starting grid search with QuanvolutionalNet, 18 combinations, 12 parallel jobs (grid_search.py:165)


Epoch 1/10, Loss: 2.3095
Epoch 2/10, Loss: 2.2887
Epoch 3/10, Loss: 2.2801
Epoch 4/10, Loss: 2.2791
Epoch 5/10, Loss: 2.2622
Epoch 6/10, Loss: 2.2523
Epoch 7/10, Loss: 2.2159
Epoch 8/10, Loss: 2.1918
Epoch 9/10, Loss: 2.1252
Epoch 10/10, Loss: 2.0605


2024-10-03 11:03:22,152 - qcml.bench.grid_search - INFO - Evaluated params: {'qkernel_shape': 2, 'classical_kernel_shape': 2, 'embedding': <function custom_iqp_embedding at 0x000002542E285480>, 'circuit': <function NQ_circuit at 0x000002542E12F250>, 'measurement': <function custom_measurement at 0x000002542E287520>, 'params': {'embedding': {'n_repeats': 2}}, 'n_classes': 10, 'learning_rate': 0.00800000037997961}, accuracy: 0.2920 (grid_search.py:241)


Epoch 1/10, Loss: 2.3030
Epoch 2/10, Loss: 2.2867
Epoch 3/10, Loss: 2.2836
Epoch 4/10, Loss: 2.2531
Epoch 5/10, Loss: 2.1901
Epoch 6/10, Loss: 2.0396
Epoch 7/10, Loss: 1.9661
Epoch 8/10, Loss: 1.8269
Epoch 9/10, Loss: 1.7511
Epoch 10/10, Loss: 1.6596


2024-10-03 11:03:27,919 - qcml.bench.grid_search - INFO - Evaluated params: {'qkernel_shape': 2, 'classical_kernel_shape': 2, 'embedding': <function custom_iqp_embedding at 0x000002542E285480>, 'circuit': <function NQ_circuit at 0x000002542E12F250>, 'measurement': <function custom_measurement at 0x000002542E287520>, 'params': {'embedding': {'n_repeats': 2}}, 'n_classes': 10, 'learning_rate': 0.029000001028180122}, accuracy: 0.3600 (grid_search.py:241)


...

2024-10-03 11:09:46,547 - qcml.bench.grid_search - INFO - Evaluated params: {'qkernel_shape': 3, 'classical_kernel_shape': 5, 'embedding': <function custom_iqp_embedding at 0x000002542E285480>, 'circuit': <function NQ_circuit at 0x000002542E12F250>, 'measurement': <function custom_measurement at 0x000002542E287520>, 'params': {'embedding': {'n_repeats': 2}}, 'n_classes': 10, 'learning_rate': 0.00800000037997961}, accuracy: 0.1960 (grid_search.py:241)


Epoch 1/10, Loss: 2.3191
Epoch 2/10, Loss: 2.2838
Epoch 3/10, Loss: 2.2973
Epoch 4/10, Loss: 2.2640
Epoch 5/10, Loss: 2.2231
Epoch 6/10, Loss: 2.3583
Epoch 7/10, Loss: 2.2007
Epoch 8/10, Loss: 2.1757
Epoch 9/10, Loss: 2.0563
Epoch 10/10, Loss: 2.2070


2024-10-03 11:10:34,632 - qcml.bench.grid_search - INFO - Evaluated params: {'qkernel_shape': 3, 'classical_kernel_shape': 5, 'embedding': <function custom_iqp_embedding at 0x000002542E285480>, 'circuit': <function NQ_circuit at 0x000002542E12F250>, 'measurement': <function custom_measurement at 0x000002542E287520>, 'params': {'embedding': {'n_repeats': 2}}, 'n_classes': 10, 'learning_rate': 0.029000001028180122}, accuracy: 0.2120 (grid_search.py:241)


Epoch 1/10, Loss: 2.3434
Epoch 2/10, Loss: 2.2937
Epoch 3/10, Loss: 2.3938
Epoch 4/10, Loss: 2.3214
Epoch 5/10, Loss: 2.2784
Epoch 6/10, Loss: 2.3322
Epoch 7/10, Loss: 2.3146
Epoch 8/10, Loss: 2.2601
Epoch 9/10, Loss: 2.2618
Epoch 10/10, Loss: 2.2581


2024-10-03 11:11:33,231 - qcml.bench.grid_search - INFO - Evaluated params: {'qkernel_shape': 3, 'classical_kernel_shape': 5, 'embedding': <function custom_iqp_embedding at 0x000002542E285480>, 'circuit': <function NQ_circuit at 0x000002542E12F250>, 'measurement': <function custom_measurement at 0x000002542E287520>, 'params': {'embedding': {'n_repeats': 2}}, 'n_classes': 10, 'learning_rate': 0.05000000074505806}, accuracy: 0.1720 (grid_search.py:241)
2024-10-03 11:11:33,238 - qcml.utils.storage - INFO - Results saved to results/grid_search/QuanvolutionalNet-quantum_mnist_IQP,_best-hypa.csv, file size: 8164 bytes (storage.py:48)


In [7]:
model_grid_classical = {
    "classical_kernel_shape": [2, 3, 5],
    "n_classes": [10],
    "use_quantum": [False],
}

gs = GridSearch(
    classifiers=[QuanvolutionalNet],
    param_grid=model_grid_classical,
    #checkpoint_interval=1,
    batch_size=50,
    experiment_name="classical_mnist,"
).run(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val)


2024-09-30 09:15:21,877 - qcml.bench.grid_search - INFO - Starting grid search with QuanvolutionalNet, 3 combinations, 50 parallel jobs (grid_search.py:165)
2024-09-30 09:15:21,880 - qcml.bench.grid_search - ERROR - Error with parameters {'classical_kernel_shape': 2, 'n_classes': 10, 'use_quantum': False}: '<=' not supported between instances of 'float' and 'NoneType' (grid_search.py:246)


TypeError: '<=' not supported between instances of 'float' and 'NoneType'